In [ ]:
import cvxpy as cvx
import numpy as np
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import math

In [49]:
def projsplx(y):
    
    m=len(y)
    bget = False
    
    s = sorted(np.array(y), reverse=True)
    tmpsum = 0.0

    for ii in range(m-1):
        tmpsum = tmpsum + s[ii]
        tmax = (tmpsum - 1.0) / (ii+1.0)
        if tmax >= s[ii+1]:
            bget = True
            break

    if bget == False:
        tmax = (tmpsum + s[m-1] - 1.0) / m

    x = np.maximum(np.array(y) - tmax, np.zeros(m))

    return x


In [50]:
def active_set(x):
    tol = 10**(-8)
    d = len(x)
    activ = np.zeros((d,d))
    val = -1
    for i in range(d):
        if x[i]>tol:
            val = i
            activ[i][i]=1
    return activ,val


In [51]:
def proj_normal_cone(y,activ,val):
    '''
    return projection onto negative normal cone of simplex
    '''
    d = len(y)
    x = cvx.Variable(d)
    obj = cvx.Minimize(cvx.sum_squares(x - y))
    if val == -1:
        constr = [x<=0]
    else:
        slack = x[val]*np.ones(d)
        constr = [x>=slack, activ@x == activ@slack]
    prob = cvx.Problem(obj, constr)
    prob.solve()
 
    return np.array(x.value).squeeze()


In [52]:
d = 10
a = np.ones(d)
y = np.random.rand(d)
y_c = projsplx(y)
print(y_c)
s = 0
for i in range(d):
    s+=y_c[i]
s

[0.         0.03956216 0.         0.24569816 0.         0.27278155
 0.41603224 0.         0.         0.02592588]


0.9999999999999999

In [53]:
x = np.array([0.3,0,0.2,0,0,0.3,0,0.1,0.1,0])
activ,val = active_set(x)
y = np.random.rand(d)
y_nc = proj_normal_cone(y,activ,val)
y_nc

array([0.40414671, 0.87119909, 0.40414671, 0.40414671, 0.8152222 ,
       0.40414671, 0.93354062, 0.40414671, 0.40414671, 0.60681141])

In [ ]:
n = 100
tol = 10**(-9)
A = np.random.rand(n,n)
A = A + A.transpose()
C = np.random.rand(n)

f = lambda x: 0.5 * np.dot(x,np.dot(A,x))-np.dot(x,C)

In [ ]:
def proj_GD(x0,niters,lr=0.001):
    x_k = np.copy(x0)
    res = []
    eta = lr
    theta = 100000
    res.append(f(x_k))

    prev_x = np.copy(x_k)
    x_k = projsplx(x_k-eta*(np.dot(A,x_k)-C))
    res.append(f(x_k))
    for i in range(niters):
        if (np.linalg.norm(np.dot(A,x_k-prev_x))) < tol:
            break
        prev_eta = eta
        eta = min(math.sqrt(1+theta)*prev_eta,np.linalg.norm(x_k-prev_x)/(2*np.linalg.norm(np.dot(A,x_k-prev_x))))

        prev_x = np.copy(x_k)
        x_k = projsplx(x_k-eta*(np.dot(A,x_k)-C))
        res.append(f(x_k))

        theta = eta/prev_eta
    
    return res


In [ ]:
def NCGD(x0,niters,lr=0.001):
    x_k = np.copy(x0)
    res = []
    eta = lr
    theta = 100000
    res.append(f(x_k))

    prev_x = np.copy(x_k)
    grad_x = np.dot(A,x_k)-C
    activ,val = active_set(grad_x)
    grad_x = grad_x - proj_normal_cone(grad_x,activ,val)
    x_k = projsplx(x_k-eta*grad_x)
    res.append(f(x_k))
    for i in range(niters):
        if (np.linalg.norm(np.dot(A,x_k-prev_x))) < tol:
            break
        prev_eta = eta
        eta = min(math.sqrt(1+theta)*prev_eta,np.linalg.norm(x_k-prev_x)/(2*np.linalg.norm(np.dot(A,x_k-prev_x))))

        prev_x = np.copy(x_k)
        grad_x = np.dot(A,x_k)-C
        activ,val = active_set(grad_x)
        grad_x = grad_x - proj_normal_cone(grad_x,activ,val)
        x_k = projsplx(x_k-eta*grad_x)
        res.append(f(x_k))

        theta = eta/prev_eta
    
    return res

In [ ]:
x0 = np.random.rand(n)
res1 = proj_GD(x0,1000)
res2 = NCGD(x0,1000)

In [ ]:
val = [res1,res2]
labels = ['GD','NCGD']
markers = ['*', 'o']
for i in range(len(val)):
    plt.plot(val[i],label = labels[i],
        marker=markers[i], markevery=20)
plt.xlabel(u'Iteration')
plt.ylabel(r'$f(x)$')
plt.legend()